In [228]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as MAE

# 01. 데이터 확인

In [52]:
feature = pd.read_table("../feature.txt", sep=' ')
feature

,FilePath,OF_d04,OF_d05,OF_d06,OF_d07,OF_d08,OF_d09,OF_d10,OF_d11,OF_d12,...,OF_d23,OF_d24,OF_d25,OF_d26,OF_d27,WhichEye,Yaw,Pitch,X,Y
0,p00/day01/0005.jpg,594,366,637,365,719,366,762,368,626,...,23.422692,524.537075,11.040978,166.869249,-27.728178,1,0.023244,-0.263194,476,758
1,p00/day01/0030.jpg,516,394,558,392,641,387,691,384,557,...,35.034721,477.888383,-123.405644,153.787666,-16.911922,0,0.213553,-0.235904,1078,693
2,p00/day01/0031.jpg,606,375,656,379,740,385,784,389,642,...,31.445826,478.976511,88.833604,130.231245,-32.639103,0,-0.158217,-0.196739,126,598
3,p00/day01/0038.jpg,594,414,642,411,723,409,768,409,639,...,46.413972,503.507272,68.867316,12.105732,-27.491372,1,0.111116,-0.026407,211,69
4,p00/day01/0063.jpg,547,370,587,366,666,360,710,357,587,...,22.173310,535.283755,-78.668260,166.500859,-20.765887,1,-0.147365,-0.252338,878,752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37662,p14/day07/0196.jpg,636,292,693,291,787,287,846,281,675,...,-10.580621,452.969811,-45.378412,2.461243,73.521260,0,0.101574,-0.004423,730,138
37663,p14/day07/0197.jpg,627,295,698,300,777,292,844,276,686,...,-10.453849,460.352050,-92.369714,133.980553,85.550006,0,0.216397,-0.240782,963,803
37664,p14/day07/0198.jpg,632,300,689,298,783,293,841,287,673,...,-9.704783,439.385058,-166.894865,132.738778,91.855893,0,0.348198,-0.230965,1339,801
37665,p14/day07/0199.jpg,631,295,690,293,784,288,844,282,674,...,-10.911425,433.764519,-170.592286,-8.092277,83.600778,0,0.336188,0.029259,1362,92


In [15]:
print("Yaw: ", contents[["Yaw"]].min().values[0], ' ~ ', contents[["Yaw"]].max().values[0])

Yaw:  -0.5277748542522531  ~  0.7064996464845319


# 02. 수식으로 yaw, pitch → x, y 변환하기
참고: https://stackoverflow.com/questions/16658529/pitch-and-yaw-to-2d-screen-coords

In [33]:
root = "../../_data/MPIIFaceGaze/"
path_people = [root+path for path in os.listdir(root) if os.path.isdir(root+path)]
path_people

['../../_data/MPIIFaceGaze/p00',
 '../../_data/MPIIFaceGaze/p01',
 '../../_data/MPIIFaceGaze/p02',
 '../../_data/MPIIFaceGaze/p03',
 '../../_data/MPIIFaceGaze/p04',
 '../../_data/MPIIFaceGaze/p05',
 '../../_data/MPIIFaceGaze/p06',
 '../../_data/MPIIFaceGaze/p07',
 '../../_data/MPIIFaceGaze/p08',
 '../../_data/MPIIFaceGaze/p09',
 '../../_data/MPIIFaceGaze/p10',
 '../../_data/MPIIFaceGaze/p11',
 '../../_data/MPIIFaceGaze/p12',
 '../../_data/MPIIFaceGaze/p13',
 '../../_data/MPIIFaceGaze/p14']

In [82]:
horiz_angle_of_view = 180
vert_angle_of_view = []
K = []

for root in path_people:
    path = root + '/Calibration/screenSize.mat'
    
    contents = sio.loadmat(path)
    w = contents["width_pixel"][0][0]
    h = contents["height_pixel"][0][0]
    
    K.append(w / horiz_angle_of_view)
    vert_angle_of_view.append(h / K[-1])

    
x_error = 0.0
y_error = 0.0

for i in range(len(feature)):
    pnum = int(feature[["FilePath"]].iloc[i].values[0][1:3])
    yaw = feature[["Yaw"]].iloc[i].values[0]
    pitch = feature[["Pitch"]].iloc[i].values[0]
    
    X = K[pnum] * (yaw + 0.5 * horiz_angle_of_view)
    Y = K[pnum] * (pitch + 0.5 * vert_angle_of_view[pnum])
    
    gt_x = feature[["X"]].iloc[i].values[0]
    gt_y = feature[["Y"]].iloc[i].values[0]
    
    x_error += abs(gt_x - X)
    y_error += abs(gt_y - Y)

print("X error: ", x_error/len(feature))
print("Y error: ", y_error/len(feature))

X error:  352.29973641357947
Y error:  219.64537725766272


horiz_angle_of_view = 20

In [84]:
horiz_angle_of_view = 20
vert_angle_of_view = []
K = []

for root in path_people:
    path = root + '/Calibration/screenSize.mat'
    
    contents = sio.loadmat(path)
    w = contents["width_pixel"][0][0]
    h = contents["height_pixel"][0][0]
    
    K.append(w / horiz_angle_of_view)
    vert_angle_of_view.append(h / K[-1])


x_error = 0.0
y_error = 0.0

for i in range(len(feature)):
    pnum = int(feature[["FilePath"]].iloc[i].values[0][1:3])
    yaw = feature[["Yaw"]].iloc[i].values[0]
    pitch = feature[["Pitch"]].iloc[i].values[0]
    
    X = K[pnum] * (yaw + 0.5 * horiz_angle_of_view)
    Y = K[pnum] * (pitch + 0.5 * vert_angle_of_view[pnum])
    
    gt_x = feature[["X"]].iloc[i].values[0]
    gt_y = feature[["Y"]].iloc[i].values[0]
    
    x_error += abs(gt_x - X)
    y_error += abs(gt_y - Y)

print("X error: ", x_error/len(feature))
print("Y error: ", y_error/len(feature))

X error:  350.8183071521226
Y error:  224.62813761386928


사람마다 error 구하기

In [99]:
horiz_angle_of_view = 180
vert_angle_of_view = []
K = []

pnum = []
x_error = []
y_error = []

for root in path_people:
    path = root + '/Calibration/screenSize.mat'
    
    contents = sio.loadmat(path)
    w = contents["width_pixel"][0][0]
    h = contents["height_pixel"][0][0]
    
    K.append(w / horiz_angle_of_view)
    vert_angle_of_view.append(h / K[-1])
    
    pnum.append(0)
    x_error.append(0.0)
    y_error.append(0.0)
    

for i in range(len(feature)):
    p_i = int(feature[["FilePath"]].iloc[i].values[0][1:3])
    yaw = feature[["Yaw"]].iloc[i].values[0]
    pitch = feature[["Pitch"]].iloc[i].values[0]
    
    X = K[p_i] * (yaw + 0.5 * horiz_angle_of_view)
    Y = K[p_i] * (pitch + 0.5 * vert_angle_of_view[p_i])
    
    gt_x = feature[["X"]].iloc[i].values[0]
    gt_y = feature[["Y"]].iloc[i].values[0]
    
    pnum[p_i] += 1
    x_error[p_i] += abs(gt_x - X)
    y_error[p_i] += abs(gt_y - Y)

In [106]:
for i, num in enumerate(pnum):
    print("p{:02d} X error: ".format(i), x_error[i]/num)
    print("p{:02d} Y error: ".format(i), y_error[i]/num)
    print()

p00 X error:  320.80001906111346
p00 Y error:  199.11899929923754

p01 X error:  359.032138907458
p01 Y error:  227.28986424569337

p02 X error:  323.35925943048574
p02 Y error:  201.19520170713994

p03 X error:  359.43909073069403
p03 Y error:  225.06195881385804

p04 X error:  319.91314930882476
p04 Y error:  197.19917833276796

p05 X error:  355.0956487145149
p05 Y error:  229.60138670746989

p06 X error:  423.9505295007138
p06 Y error:  261.97783342832275

p07 X error:  395.0538021555432
p07 Y error:  238.2704047523539

p08 X error:  356.37954941325336
p08 Y error:  225.46568401265404

p09 X error:  364.93857617819816
p09 Y error:  219.27402716073186

p10 X error:  358.22637541277084
p10 Y error:  228.13654997370384

p11 X error:  314.85418702611156
p11 Y error:  199.85399592345226

p12 X error:  317.7643568980098
p12 Y error:  198.4633162938985

p13 X error:  321.4705548643033
p13 Y error:  200.7755726207982

p14 X error:  366.00537878782626
p14 Y error:  226.2590076193765



In [110]:
for i, num in enumerate(pnum):
    print("p{:02d}: ".format(i), num)

p00:  2927
p01:  2904
p02:  2916
p03:  2929
p04:  2860
p05:  2870
p06:  2877
p07:  2843
p08:  2767
p09:  2719
p10:  2194
p11:  2262
p12:  1601
p13:  1498
p14:  1500


# 03. 모델로 yaw, pitch → x, y 변환하기
사람마다 돌려보기,,

In [186]:
def loadAllData(path):
    contents = pd.read_table(path, sep=' ').sort_values(by="FilePath")
    
    """
        string:
            contents.iloc[:, 0]     # file path
        int:
            contents.iloc[:, 1:13]  # original feature D05 ~ D15
            contents.iloc[:, -5]    # original feature D28 (which eye)
            contents.iloc[:, -2:]   # original feature D02 ~ D03 (target x, y)
        float:
            contents.iloc[:, 13:-5] # original feature D16 ~ D27
            contents.iloc[:, -4:-2] # yaw, pitch
    """
    contents.iloc[:, 1:13] = contents.iloc[:, 1:13].values.astype("int32")
    contents.iloc[:, -5] = contents.iloc[:, -5].values.astype("int32")
    contents.iloc[:, -2:] = contents.iloc[:, -2:].values.astype("int32")
    contents.iloc[:, 13:-5] = contents.iloc[:, 13:-5].values.astype("float32")
    contents.iloc[:, -4:-2] = contents.iloc[:, -4:-2].values.astype("float32")
    
    return contents

In [243]:
def loadData(contents, p_idx):
    np_contents = np.array(contents)    
    p_idx = "p{:02d}".format(p_idx)
    
    i_start = -1
    i_end = -1
    flag = 0
    
    for i in range(len(contents)):
        index = np_contents[i][0].split('/')[0]
        if not flag and p_idx == index:
            flag = 1
            i_start = i
        elif flag and p_idx != index:
            i_end = i
            break
    
    train = np.concatenate((np_contents[:i_start], np_contents[i_end:]), axis=0)
    test = np_contents[i_start:i_end]
    
    # 상관계수 상관없이 전부 때려박기
#     train_X = train[:, 1:-2]
#     test_X = test[:, 1:-2]
    
    # 상관계수 뽑은 후 관련된 feature
    train_X = np.concatenate((train[:, -8:-5], train[:, -4:-2]), axis=1)
    test_X = np.concatenate((test[:, -8:-5], test[:, -4:-2]), axis=1)

    train_Y = train[:, -2:]
    test_Y = test[:, -2:]
        
    return train_X, train_Y, test_X, test_Y

In [244]:
for i in range(15):
    train_X, train_Y, test_X, test_Y = loadData(contents, i, 'x')
    
    model = LinearRegression()
    model.fit(train_X, train_Y)
    
    print("=============== p{:02d} ===============".format(i))
    print("Test MAE: {:.4f}".format(MAE(test_Y, model.predict(test_X))))
    print()

=============== p00 ===============
Train R2: 0.9666
Train MAE: 48.2745

Test R2: 0.8914
Test MAE: 98.6502

=============== p01 ===============
Train R2: 0.9620
Train MAE: 51.1693

Test R2: 0.9901
Test MAE: 26.5865

=============== p02 ===============
Train R2: 0.9645
Train MAE: 48.1861

Test R2: 0.9549
Test MAE: 57.7573

=============== p03 ===============
Train R2: 0.9621
Train MAE: 50.1940

Test R2: 0.9863
Test MAE: 33.2012

=============== p04 ===============
Train R2: 0.9626
Train MAE: 51.1755

Test R2: 0.9889
Test MAE: 21.8020

=============== p05 ===============
Train R2: 0.9655
Train MAE: 48.7097

Test R2: 0.9406
Test MAE: 73.2554

=============== p06 ===============
Train R2: 0.9660
Train MAE: 45.8519

Test R2: 0.9418
Test MAE: 88.8564

=============== p07 ===============
Train R2: 0.9611
Train MAE: 51.3359

Test R2: 0.9944
Test MAE: 23.0920

=============== p08 ===============
Train R2: 0.9659
Train MAE: 47.8044

Test R2: 0.9352
Test MAE: 82.4996

=============== p09 ========

좌표 x, y 따로 돌려보기......

In [266]:
def loadData(contents, p_idx, xy):
    np_contents = np.array(contents)    
    p_idx = "p{:02d}".format(p_idx)
    
    i_start = -1
    i_end = -1
    flag = 0
    
    for i in range(len(contents)):
        index = np_contents[i][0].split('/')[0]
        if not flag and p_idx == index:
            flag = 1
            i_start = i
        elif flag and p_idx != index:
            i_end = i
            break
    
    if i_end == -1:
        i_end = len(contents)
        
    train = np.concatenate((np_contents[:i_start], np_contents[i_end:]), axis=0)
    test = np_contents[i_start:i_end]
    
    # 상관계수 상관없이 전부 때려박기
#     train_X = train[:, 1:-2]
#     test_X = test[:, 1:-2]
    
    # 상관계수 뽑은 후 관련된 feature
    train_X = np.concatenate((train[:, -8:-5], train[:, -4:-2]), axis=1)
    test_X = np.concatenate((test[:, -8:-5], test[:, -4:-2]), axis=1)
    
    if xy == 'x':
        train_Y = train[:, -2]
        test_Y = test[:, -2]
    elif xy == 'y':
        train_Y = train[:, -1]
        test_Y = test[:, -1]        
        
    return train_X, train_Y, test_X, test_Y

X

In [267]:
for i in range(15):
    train_X, train_Y, test_X, test_Y = loadData(contents, i, 'x')
    
    model = LinearRegression()
    model.fit(train_X, train_Y)
    
    print("=============== p{:02d} ===============".format(i))
    print("Test MAE: {:.4f}".format(MAE(test_Y, model.predict(test_X))))
    print()

=============== p00 ===============
Test MAE: 136.9667

=============== p01 ===============
Test MAE: 26.8487

=============== p02 ===============
Test MAE: 57.5830

=============== p03 ===============
Test MAE: 50.8903

=============== p04 ===============
Test MAE: 16.9989

=============== p05 ===============
Test MAE: 134.9908

=============== p06 ===============
Test MAE: 152.1225

=============== p07 ===============
Test MAE: 31.1172

=============== p08 ===============
Test MAE: 106.6903

=============== p09 ===============
Test MAE: 61.8945

=============== p10 ===============
Test MAE: 47.3962

=============== p11 ===============
Test MAE: 30.8826

=============== p12 ===============
Test MAE: 28.3431

=============== p13 ===============
Test MAE: 118.8248

=============== p14 ===============
Test MAE: 394.1556



Y

In [268]:
for i in range(15):
    train_X, train_Y, test_X, test_Y = loadData(contents, i, 'y')
    
    model = LinearRegression()
    model.fit(train_X, train_Y)
    
    print("=============== p{:02d} ===============".format(i))
    print("Test MAE: {:.4f}".format(MAE(test_Y, model.predict(test_X))))
    print()

=============== p00 ===============
Test MAE: 60.3337

=============== p01 ===============
Test MAE: 26.3243

=============== p02 ===============
Test MAE: 57.9316

=============== p03 ===============
Test MAE: 15.5121

=============== p04 ===============
Test MAE: 26.6050

=============== p05 ===============
Test MAE: 11.5199

=============== p06 ===============
Test MAE: 25.5903

=============== p07 ===============
Test MAE: 15.0667

=============== p08 ===============
Test MAE: 58.3089

=============== p09 ===============
Test MAE: 12.8956

=============== p10 ===============
Test MAE: 12.0791

=============== p11 ===============
Test MAE: 39.4067

=============== p12 ===============
Test MAE: 11.2005

=============== p13 ===============
Test MAE: 49.4722

=============== p14 ===============
Test MAE: 89.0791

